In [7]:
# import numpy as np
import pandas as pd
import pprint
from scipy import stats

class BidirectionalHeteroAssociative(object):
    def __init__(self, data, treshold):
        self.train_data = []
        self.treshold = treshold
        for item in data:
            self.train_data.append(
                [self.make_binary(item[0]),
                 self.make_binary(item[1])]
                )
        print('Train Data:')
        for i in range(len(self.train_data)):
            print(self.train_data[i][0],self.train_data[i][1],sep=" ")
        self.len_x = len(self.train_data[0][1])
        self.len_y = len(self.train_data[0][0])
        self.weight = [[0 for x in range(self.len_x)] for x in range(self.len_y)]
        self.train_association()

    def train_association(self):
        for assoc_pair in self.train_data:
            X = assoc_pair[0]
            Y = assoc_pair[1]
            for i, xi in enumerate(X):
                for j, yi in enumerate(Y):
                    self.weight[i][j] = self.weight[i][j] + (xi * yi)

    def test_association(self, test_data):
        matriks_result = [0] * self.len_x
        test_data = self.calculate_threshold(test_data)
        print('Bobot Test:')
        for x in range(self.len_x):
            for y in range(self.len_y):
                matriks_result[x] = matriks_result[x] + (test_data[y] * self.weight[y][x])
                print(matriks_result[x],end=" ")
            print()
        return self.calculate_threshold(matriks_result)

    def get_weight_matrix(self):
        return self.weight

    def calculate_threshold(self, vector):
        print('calculate')
        print([1 if element > self.treshold else element if element == self.treshold else -1 for element in vector])
        return [1 if element > self.treshold else element if element == self.treshold else -1 for element in vector]
    
    def make_binary(self, vector):
        print('binary')
        print([1 if element > self.treshold else element if element == self.treshold else -1 for element in vector])
        return [1 if element > self.treshold else element if element == self.treshold else -1 for element in vector]

        #return [1 if element > 0 else 0 if element == 0 else -1 for element in vector]
#         return [1 if element > 0 else 0 for element in vector]
    
    def normalized(self,data):
        return stats.zscore(data)

#without norm
# data_pairs = [
#     [[1068180602,0.05521987,0.36574934,0.9423917,201117057,0.00351113],[0,1,0]],
# [[997424749,0.05997117,0.37067181,0.93518036,199555184,0.0040403],[0,1,0]],
# [[1032424749,0.05697098,0.36919299,0.93953386,205075808,0.00360527],[0,1,0]],
# [[1116855072,0.05389023,0.35931052,0.94398995,209176143,0.00327491],[0,1,0]],
# [[909402453,0.06062716,0.38863038,0.94702461,189570792,0.00410939],[0,1,0]],
# [[1017703.456,0.05968909,0.36400829,0.92928882,201849498,0.00403872],[0,1,0]],
# [[972132664,0.0593779,0.37760417,0.93840413,202362319,0.00391612],[0,1,0]],
# [[1048334448,0.05886013,0.3642354,0.93363917,202241918,0.00394371],[0,1,0]],
# [[101899777,0.05759129,0.36915872,0.93801908,200483835,0.00376701],[0,1,0]],
# [[953204013,0.05870078,0.38111105,0.94277563,191670011,0.00391016],[0,1,0]],
# [[993405797,0.05822474,0.3715318,0.94121394,209392419,0.00370532],[0,0,1]],
# [[875852843,0.06742338,0.39042761,0.92887682,195053512,0.00495384],[0,0,1]],
# [[815685619,0.06778131,0.40330452,0.93537207,181975474,0.0050745],[0,0,1]],
# [[1037973244,0.05857664,0.3709749,0.93860601,208316611,0.00371578],[0,0,1]],
# [[1051491639,0.05718544,0.36990652,0.9412644,21374359,0.00355955],[0,0,1]],
# [[918240803,0.06432971,0.38383289,0.93280868,19661427,0.00455201],[0,0,1]],
# [[878956522,0.06480552,0.39210451,0.93751291,189154961,0.00464208],[0,0,1]],
# [[813149387,0.06831331,0.39942319,0.93364619,18519175,0.00504401],[0,0,1]],
# [[949405797,0.06172657,0.37577589,0.93417195,197694537,0.00429284],[0,0,1]],
# [[975843924,0.06265329,0.37593527,0.9302802,199523969,0.00438733],[0,0,1]],
# [[702244147,0.06373407,0.42028533,0.95582566,172733556,0.00447338],[0,1,1]],
# [[681536232,0.06382992,0.426039,0.95902441,168369008,0.00452387],[0,1,1]],
# [[686833891,0.06424293,0.42788867,0.95892024,170128205,0.00447364],[0,1,1]],
# [[561575251,0.07354432,0.46605064,0.96407646,14154961,0.00607186],[0,1,1]],
# [[974537347,0.05681873,0.38499769,0.95768986,211956522,0.00348419],[0,1,1]],
# [[654005574,0.06726295,0.42221438,0.95115411,166123746,0.0050031],[0,1,1]],
# [[697975474,0.06536422,0.43235462,0.95601524,164023411,0.00469729],[0,1,1]],
# [[755811594,0.06171216,0.41237728,0.95529909,166023411,0.00441443],[0,1,1]],
# [[627283166,0.07017439,0.44806876,0.95655877,155114827,0.00545981],[0,1,1]]
# ]

#with norm fitur 1 dan 5
data_pairs = [
    [[0.952042745,0.05521987,0.36574934,0.9423917,0.945200306,0.00351113],[0,1,0]],
[[0.882329474,0.05997117,0.37067181,0.93518036,0.937304145,0.0040403],[0,1,0]],
[[0.916813752,0.05697098,0.36919299,0.93953386,0.965214056,0.00360527],[0,1,0]],
[[1,0.05389023,0.35931052,0.94398995,0.985943594,0.00327491],[0,1,0]],
[[0.795604181,0.06062716,0.38863038,0.94702461,0.886827334,0.00410939],[0,1,0]],
[[0.902309376,0.05968909,0.36400829,0.92928882,0.948903214,0.00403872],[0,1,0]],
[[0.857410067,0.0593779,0.37760417,0.93840413,0.951495818,0.00391612],[0,1,0]],
[[0.932489023,0.05886013,0.3642354,0.93363917,0.950887122,0.00394371],[0,1,0]],
[[0,0.05759129,0.36915872,0.93801908,0.941999007,0.00376701],[0,1,0]],
[[0.838760328,0.05870078,0.38111105,0.94277563,0.897440086,0.00391016],[0,1,0]],
[[0.878369741,0.05822474,0.3715318,0.94121394,0.987036993,0.00370532],[0,0,1]],
[[0.762548922,0.06742338,0.39042761,0.92887682,0.914545619,0.00495384],[0,0,1]],
[[0.703268258,0.06778131,0.40330452,0.93537207,0.848428658,0.0050745],[0,0,1]],
[[0.92228049,0.05857664,0.3709749,0.93860601,0.981598168,0.00371578],[0,0,1]],
[[0.935599693,0.05718544,0.36990652,0.9412644,0.036498185,0.00355955],[0,0,1]],
[[0.804312298,0.06432971,0.38383289,0.93280868,0.027838334,0.00455201],[0,0,1]],
[[0.765606868,0.06480552,0.39210451,0.93751291,0.88472507,0.00464208],[0,0,1]],
[[0.700769397,0.06831331,0.39942319,0.93364619,0.022063597,0.00504401],[0,0,1]],
[[0.835018078,0.06172657,0.37577589,0.93417195,0.92789751,0.00429284],[0,0,1]],
[[0.861066641,0.06265329,0.37593527,0.9302802,0.937146335,0.00438733],[0,0,1]],
[[0.591498338,0.06373407,0.42028533,0.95582566,0.801705478,0.00447338],[0,1,1]],
[[0.571095553,0.06382992,0.426039,0.95902441,0.779640192,0.00452387],[0,1,1]],
[[0.576315151,0.06424293,0.42788867,0.95892024,0.788533939,0.00447364],[0,1,1]],
[[0.452902188,0.07354432,0.46605064,0.96407646,0,0.00607186],[0,1,1]],
[[0.859779317,0.05681873,0.38499769,0.95768986,1,0.00348419],[0,1,1]],
[[0.543970557,0.06726295,0.42221438,0.95115411,0.768289109,0.0050031],[0,1,1]],
[[0.587292563,0.06536422,0.43235462,0.95601524,0.757670714,0.00469729],[0,1,1]],
[[0.644276472,0.06171216,0.41237728,0.95529909,0.767781858,0.00441443],[0,1,1]],
[[0.517641902,0.07017439,0.44806876,0.95655877,0.712632728,0.00545981],[0,1,1]]
]

b = BidirectionalHeteroAssociative(data_pairs,0.4)
print ('\n')
print ('Matrix Bobot: ')
for i,angka in enumerate(b.get_weight_matrix()):
    for j in range(len(b.get_weight_matrix()[i])):
        print(b.get_weight_matrix()[i][j],end=" ")
    print()
print ('\n')
print ('[0.703268258,0.06778131,0.40330452,0.93537207,0.848428658,0.0050745] ---> ', b.test_association([0.703268258,0.06778131,0.40330452,0.93537207,0.848428658,0.0050745]))
print("------------------------------------------------------------------------------------------")
print ('[0.878369741,0.05822474,0.3715318,0.94121394,0.987036993,0.00370532] ---> ', b.test_association([0.878369741,0.05822474,0.3715318,0.94121394,0.987036993,0.00370532]))
print("------------------------------------------------------------------------------------------")
print ('[0.517641902,0.07017439,0.44806876,0.95655877,0.712632728,0.00545981] ---> ', b.test_association([0.517641902,0.07017439,0.44806876,0.95655877,0.712632728,0.00545981]))
print("------------------------------------------------------------------------------------------")
print ('[0,1,1,1,1,1] ---> ', b.test_association([0,1,1,1,1,1]))


binary
[1, -1, -1, 1, 1, -1]
binary
[-1, 1, -1]
binary
[1, -1, -1, 1, 1, -1]
binary
[-1, 1, -1]
binary
[1, -1, -1, 1, 1, -1]
binary
[-1, 1, -1]
binary
[1, -1, -1, 1, 1, -1]
binary
[-1, 1, -1]
binary
[1, -1, -1, 1, 1, -1]
binary
[-1, 1, -1]
binary
[1, -1, -1, 1, 1, -1]
binary
[-1, 1, -1]
binary
[1, -1, -1, 1, 1, -1]
binary
[-1, 1, -1]
binary
[1, -1, -1, 1, 1, -1]
binary
[-1, 1, -1]
binary
[-1, -1, -1, 1, 1, -1]
binary
[-1, 1, -1]
binary
[1, -1, -1, 1, 1, -1]
binary
[-1, 1, -1]
binary
[1, -1, -1, 1, 1, -1]
binary
[-1, -1, 1]
binary
[1, -1, -1, 1, 1, -1]
binary
[-1, -1, 1]
binary
[1, -1, 1, 1, 1, -1]
binary
[-1, -1, 1]
binary
[1, -1, -1, 1, 1, -1]
binary
[-1, -1, 1]
binary
[1, -1, -1, 1, -1, -1]
binary
[-1, -1, 1]
binary
[1, -1, -1, 1, -1, -1]
binary
[-1, -1, 1]
binary
[1, -1, -1, 1, 1, -1]
binary
[-1, -1, 1]
binary
[1, -1, -1, 1, -1, -1]
binary
[-1, -1, 1]
binary
[1, -1, -1, 1, 1, -1]
binary
[-1, -1, 1]
binary
[1, -1, -1, 1, 1, -1]
binary
[-1, -1, 1]
binary
[1, -1, 1, 1, 1, -1]
binary
[-

In [2]:
class BAM(object):
    def __init__(self, data):
        self.AB = []
        # store associations in bipolar form to the array
        for item in data:
            self.AB.append(
                [self.__l_make_bipolar(item[0]),
                 self.__l_make_bipolar(item[1])]
                )
        self.len_x = len(self.AB[0][1])
        self.len_y = len(self.AB[0][0])
        print(self.AB)

        # create empty BAM matrix
        self.M = [[0 for x in range(self.len_x)] for x in range(self.len_y)]
        # compute BAM matrix from associations
        self.__create_bam()

    def __create_bam(self):
        '''Bidirectional associative memory'''
        for assoc_pair in self.AB:
          X = assoc_pair[0]
          Y = assoc_pair[1]
         # calculate M
          for idx, xi in enumerate(X):
            for idy, yi in enumerate(Y):
              self.M[idx][idy] += xi * yi

    def get_assoc(self, A):
        '''Return association for input vector A'''
        A = self.__mult_mat_vec(A)
        return self.__threshold(A)

    def get_bam_matrix(self):
        '''Return BAM matrix'''
        return self.M

    def __mult_mat_vec(self, vec):
        '''Multiply imput vector with BAM matrix'''
        v_res = [0] * self.len_x
        for x in range(self.len_x):
            for y in range(self.len_y):
                v_res[x] += vec[y] * self.M[y][x]
        return v_res

    def __threshold(self, vec):
        '''Transform vector to [0, 1]'''
        ret_vec = []
        for i in vec:
            if i < 0:
                ret_vec.append(0)
            else:
                ret_vec.append(1)
        return ret_vec
    
    def __l_make_bipolar(self, vec):
        '''Transform vector to bipolar form [-1, 1]'''
        ret_vec = []
        for item in vec:
            if item == 0:
                ret_vec.append(-1)
            else:
                ret_vec.append(1)
        return ret_vec

    
    
if __name__ == "__main__":
#     data_pairs  = [
#         [[1, 0, 1, 0, 1, 0], [1, 1, 0, 0]],
#         [[1, 1, 1, 0, 0, 0], [1, 0, 1, 0]]
#         ]


# with norm fitur 1 dan 5
    data_pairs = [
    [[0.952042745,0.05521987,0.36574934,0.9423917,0.945200306,0.00351113],[0,1,0,0,1]],
    [[0.861066641,0.06265329,0.37593527,0.9302802,0.937146335,0.00438733],[0,0,1,0,1]],
    [[0.517641902,0.07017439,0.44806876,0.95655877,0.712632728,0.00545981],[0,1,1,0,1]]
    ]
    b = BAM(data_pairs)

    import pprint
    pp = pprint.PrettyPrinter(indent=4)
    print ('Matrix: ')
    pp.pprint(b.get_bam_matrix())
    print ('\n')
    print ('[1, 0, 1, 0, 1, 0] ---> ', b.get_assoc([-1, -1, 1, -1, 1, 1]))
    print ('[1, 1, 1, 0, 0, 0] ---> ', b.get_assoc([0.517641902,0.07017439,0.44806876,0.95655877,0.712632728,0.00545981]))

[[[1, 1, 1, 1, 1, 1], [-1, 1, -1, -1, 1]], [[1, 1, 1, 1, 1, 1], [-1, -1, 1, -1, 1]], [[1, 1, 1, 1, 1, 1], [-1, 1, 1, -1, 1]]]
Matrix: 
[   [-3, 1, 1, -3, 3],
    [-3, 1, 1, -3, 3],
    [-3, 1, 1, -3, 3],
    [-3, 1, 1, -3, 3],
    [-3, 1, 1, -3, 3],
    [-3, 1, 1, -3, 3]]


[1, 0, 1, 0, 1, 0] --->  [1, 1, 1, 1, 1]
[1, 1, 1, 0, 0, 0] --->  [0, 1, 1, 0, 1]
